In [2]:
import pandas as pd
import plotly.graph_objects as go
from dash import Dash, html, dcc, Input, Output, exceptions
import numpy as np

# Load your data
df_radar = pd.read_csv("CleanedData/player_radar.csv")
df_stats = pd.read_csv("CleanedData/player_stats_cleaned.csv")

# Merge dataframes on 'player' after normalizing the 'player' column
df_merged = pd.merge(df_radar, df_stats, on='player', how='outer')

# Calculate performance metrics
df_merged['passing_commulative_performance'] = (
    0.05 * df_merged['passes_completed'] +
    1.5 * df_merged['assists_x'] +
    1.5 * df_merged['assisted_shots'] +
    0.2 * df_merged['passes_into_final_third'] +
    0.4 * df_merged['passes_into_penalty_area'] +
    0.3 * df_merged['crosses_into_penalty_area'] +
    0.1 * df_merged['progressive_passes']
)

df_merged['shooting_commulative_performance'] = (
    3 * df_merged['goals_x'] +
    0.3 * df_merged['shots_on_target'] +
    0.1 * df_merged['shots'] +
    0.2 * df_merged['shots_free_kicks'] +
    df_merged['pens_made_x']
)

df_merged['defence_commulative_performance'] = (
    -0.2 * df_merged['dribbled_past'] -
    df_merged['errors'] +
    0.5 * df_merged['blocks'] +
    0.75 * df_merged['blocked_shots'] +
    0.75 * df_merged['blocked_passes'] +
    df_merged['tackles_interceptions'] +
    0.2 * df_merged['clearances']
)

df_merged['misc_commulative_performance'] = (
    -0.25 * df_merged['cards_yellow'] -
    1 * df_merged['cards_red'] -
    0.1 * df_merged['fouls'] +
    0.25 * df_merged['interceptions_x'] +
    0.3 * df_merged['tackles_won_x'] +
    1.5 * df_merged['pens_won'] -
    1.5 * df_merged['pens_conceded'] +
    0.25 * df_merged['ball_recoveries'] -
    3 * df_merged['own_goals'] -
    0.25 * df_merged['aerials_lost'] +
    0.25 * df_merged['aerials_won']
)

app = Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='team-dropdown',
        options=[{'label': 'No Team', 'value': 'No Team'}] + [{'label': team, 'value': team} for team in df_merged['team_x'].dropna().unique()],
        placeholder="Select a team",
        clearable=False
    ),
    dcc.Dropdown(
        id='player-dropdown',
        placeholder="Select players",
        multi=True,
        clearable=False
    ),
    dcc.Dropdown(
        id='highlight-player-dropdown',
        placeholder="Select a player to highlight",
        clearable=True
    ),
    dcc.Graph(id='radar-chart', style={"height": "90vh"})  # Adjust the height of the radar chart
])

@app.callback(
    Output('player-dropdown', 'options'),
    Output('player-dropdown', 'value'),
    Input('team-dropdown', 'value')
)
def set_player_options(selected_team):
    if not selected_team or selected_team == 'No Team':
        player_options = [{'label': player, 'value': player} for player in df_merged['player'].unique()]
        return player_options, []
    filtered_df = df_merged[df_merged['team_x'] == selected_team]
    player_options = [{'label': player, 'value': player} for player in filtered_df['player'].unique()]
    return player_options, [option['value'] for option in player_options]

@app.callback(
    Output('highlight-player-dropdown', 'options'),
    Input('player-dropdown', 'value')
)
def set_highlight_player_options(selected_players):
    if not selected_players:
        return []
    return [{'label': player, 'value': player} for player in selected_players]

@app.callback(
    Output('radar-chart', 'figure'),
    [Input('team-dropdown', 'value'),
     Input('player-dropdown', 'value'),
     Input('highlight-player-dropdown', 'value')]
)
def update_radar_chart(selected_team, selected_players, highlighted_player):
    if not selected_players:
        return go.Figure()

    if selected_team == 'No Team':
        filtered_df = df_merged
    else:
        filtered_df = df_merged[df_merged['team_x'] == selected_team]

    categories = [
        'defence_commulative_performance',
        'passing_commulative_performance',
        'shooting_commulative_performance',
        'misc_commulative_performance',
        'defence_commulative_performance'  # Repeat the first category to close the radar chart loop
    ]
    categories_labels = ['Defence', 'Passing', 'Shooting', 'Miscellaneous', 'Defence']

    radial_ticks = np.arange(0, 60, 2)  # Adjust tick values

    fig = go.Figure()

    for player in selected_players:
        player_data = filtered_df[filtered_df['player'] == player]
        if not player_data.empty:
            fig.add_trace(go.Scatterpolar(
                r=[player_data[cat].values[0] for cat in categories],
                theta=categories_labels,
                name=player,
                mode='lines',
                line_color='green' if player == highlighted_player else 'grey',
                line_shape='spline',
                line_smoothing=0.8,
                opacity=1 if player == highlighted_player else 0.6,
                line_width=3 if player == highlighted_player else 1
            ))

    fig.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, tickvals=radial_ticks),
            bgcolor='white',
            radialaxis_tickfont_color='darkgrey',
            angularaxis_color='grey',
            angularaxis_showline=False,
            radialaxis_showline=False,
            radialaxis_gridcolor='#F2F2F2'
        ),
        title="Performance Metrics",
        font=dict(size=15)
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)


OSError: Address 'http://127.0.0.1:8051' already in use.
    Try passing a different port to run_server.

In [ ]:
import threading
from IPython.display import display, HTML

# Function to run the Dash app
def run_dash():
    app.run_server(debug=False, use_reloader=False)

# Start the Dash app in a separate thread
threading.Thread(target=run_dash).start()

# Display the link to open the app in the browser
display(HTML(f"""
    <a href="http://127.0.0.1:8051/" target="_blank">
        Open Dash App
    </a>
"""))